In [1]:
#! /usr/bin/env python
# coding=utf-8
#================================================================
#   Copyright (C) 2019 * Ltd. All rights reserved.
#
#   Editor      : VIM
#   File name   : evaluate.py
#   Author      : YunYang1994
#   Created date: 2019-02-21 15:30:26
#   Description :
#
#================================================================

import cv2
import os
import shutil
import numpy as np
import tensorflow as tf
import core.utils as utils
from core.config import cfg
from core.yolov3 import YOLOV3

class YoloTest(object):
    def __init__(self):
        self.input_size       = cfg.TEST.INPUT_SIZE
        self.anchor_per_scale = cfg.YOLO.ANCHOR_PER_SCALE
        self.classes          = utils.read_class_names(cfg.YOLO.CLASSES)
        self.num_classes      = len(self.classes)
        self.anchors          = np.array(utils.get_anchors(cfg.YOLO.ANCHORS))
        self.score_threshold  = cfg.TEST.SCORE_THRESHOLD
        self.iou_threshold    = cfg.TEST.IOU_THRESHOLD
        self.moving_ave_decay = cfg.YOLO.MOVING_AVE_DECAY
        self.annotation_path  = cfg.TEST.ANNOT_PATH
        self.weight_file      = cfg.TEST.WEIGHT_FILE
        self.write_image      = cfg.TEST.WRITE_IMAGE
        self.write_image_path = cfg.TEST.WRITE_IMAGE_PATH
        self.show_label       = cfg.TEST.SHOW_LABEL

        with tf.name_scope('input'):
            self.input_data = tf.placeholder(dtype=tf.float32, name='input_data')
            self.trainable  = tf.placeholder(dtype=tf.bool,    name='trainable')

        model = YOLOV3(self.input_data, self.trainable)
        self.pred_sbbox, self.pred_mbbox, self.pred_lbbox = model.pred_sbbox, model.pred_mbbox, model.pred_lbbox

        with tf.name_scope('ema'):
            ema_obj = tf.train.ExponentialMovingAverage(self.moving_ave_decay)

        self.sess  = tf.Session(config=tf.ConfigProto(allow_soft_placement=True))
        self.saver = tf.train.Saver(ema_obj.variables_to_restore())
        self.saver.restore(self.sess, self.weight_file)
    def predict(self, image):

        org_image = np.copy(image)
        org_h, org_w, _ = org_image.shape

        image_data = utils.image_preporcess(image, [self.input_size, self.input_size])
        image_data = image_data[np.newaxis, ...]

        pred_sbbox, pred_mbbox, pred_lbbox = self.sess.run(
            [self.pred_sbbox, self.pred_mbbox, self.pred_lbbox],
            feed_dict={
                self.input_data: image_data,
                self.trainable: False
            }
        )

        pred_bbox = np.concatenate([np.reshape(pred_sbbox, (-1, 5 + self.num_classes)),
                                    np.reshape(pred_mbbox, (-1, 5 + self.num_classes)),
                                    np.reshape(pred_lbbox, (-1, 5 + self.num_classes))], axis=0)
        bboxes = utils.postprocess_boxes(pred_bbox, (org_h, org_w), self.input_size, self.score_threshold)
        bboxes = utils.nms(bboxes, self.iou_threshold)

        return bboxes

    def evaluate(self):
        predicted_dir_path = './mAP/predicted'
        ground_truth_dir_path = './mAP/ground-truth'
        if os.path.exists(predicted_dir_path): shutil.rmtree(predicted_dir_path)
        if os.path.exists(ground_truth_dir_path): shutil.rmtree(ground_truth_dir_path)
        if os.path.exists(self.write_image_path): shutil.rmtree(self.write_image_path)
        os.mkdir(predicted_dir_path)
        os.mkdir(ground_truth_dir_path)
        os.mkdir(self.write_image_path)

        with open(self.annotation_path, 'r') as annotation_file:
            for num, line in enumerate(annotation_file):
                annotation = line.strip().split()
                image_path = annotation[0]
                image_name = image_path.split('/')[-1]
                image = cv2.imread(image_path)
                bbox_data_gt = np.array([list(map(int, box.split(','))) for box in annotation[1:]])

                if len(bbox_data_gt) == 0:
                    bboxes_gt=[]
                    classes_gt=[]
                else:
                    bboxes_gt, classes_gt = bbox_data_gt[:, :4], bbox_data_gt[:, 4]
                ground_truth_path = os.path.join(ground_truth_dir_path, str(num) + '.txt')

                print('=> ground truth of %s:' % image_name)
                num_bbox_gt = len(bboxes_gt)
                with open(ground_truth_path, 'w') as f:
                    for i in range(num_bbox_gt):
                        class_name = self.classes[classes_gt[i]]
                        xmin, ymin, xmax, ymax = list(map(str, bboxes_gt[i]))
                        bbox_mess = ' '.join([class_name, xmin, ymin, xmax, ymax]) + '\n'
                        f.write(bbox_mess)
                        print('\t' + str(bbox_mess).strip())
                print('=> predict result of %s:' % image_name)
                predict_result_path = os.path.join(predicted_dir_path, str(num) + '.txt')
                bboxes_pr = self.predict(image)

                if self.write_image:
                    image = utils.draw_bbox(image, bboxes_pr, show_label=self.show_label)
                    cv2.imwrite(self.write_image_path+image_name, image)

                with open(predict_result_path, 'w') as f:
                    for bbox in bboxes_pr:
                        coor = np.array(bbox[:4], dtype=np.int32)
                        score = bbox[4]
                        class_ind = int(bbox[5])
                        class_name = self.classes[class_ind]
                        score = '%.4f' % score
                        xmin, ymin, xmax, ymax = list(map(str, coor))
                        bbox_mess = ' '.join([class_name, score, xmin, ymin, xmax, ymax]) + '\n'
                        f.write(bbox_mess)
                        print('\t' + str(bbox_mess).strip())

    def voc_2012_test(self, voc2012_test_path):

        img_inds_file = os.path.join(voc2012_test_path, 'ImageSets', 'Main', 'test.txt')
        with open(img_inds_file, 'r') as f:
            txt = f.readlines()
            image_inds = [line.strip() for line in txt]

        results_path = 'results/VOC2012/Main'
        if os.path.exists(results_path):
            shutil.rmtree(results_path)
        os.makedirs(results_path)

        for image_ind in image_inds:
            image_path = os.path.join(voc2012_test_path, 'JPEGImages', image_ind + '.jpg')
            image = cv2.imread(image_path)

            print('predict result of %s:' % image_ind)
            bboxes_pr = self.predict(image)
            for bbox in bboxes_pr:
                coor = np.array(bbox[:4], dtype=np.int32)
                score = bbox[4]
                class_ind = int(bbox[5])
                class_name = self.classes[class_ind]
                score = '%.4f' % score
                xmin, ymin, xmax, ymax = list(map(str, coor))
                bbox_mess = ' '.join([image_ind, score, xmin, ymin, xmax, ymax]) + '\n'
                with open(os.path.join(results_path, 'comp4_det_test_' + class_name + '.txt'), 'a') as f:
                    f.write(bbox_mess)
                print('\t' + str(bbox_mess).strip())


if __name__ == '__main__': YoloTest().evaluate()





Instructions for updating:
Colocations handled automatically by placer.
Instructions for updating:
Use keras.layers.batch_normalization instead.
Instructions for updating:
Use standard file APIs to check for files with this prefix.
INFO:tensorflow:Restoring parameters from ./checkpoint/yolov3_test_loss=22.1229.ckpt-341
=> ground truth of 2009041517.jpg:
	bowl 65 190 100 223
	plastic_bag 108 90 242 201
	plastic_bag 77 1 189 75
	plastic_bag 21 13 84 81
	plastic_bag 9 75 85 136
	plastic_bag 182 34 257 113
	plastic_bag 186 1 247 38
	plastic_bag 298 162 356 208
	plastic_bag 306 210 339 245
=> predict result of 2009041517.jpg:
	tire 0.6671 109 60 251 202
	bowl 0.8119 241 32 280 71
	bowl 0.3395 64 189 100 222
	bucket 0.3194 13 68 84 133
	plastic_bag 0.9266 113 80 249 202
	plastic_bag 0.5695 13 70 85 138
	plastic_bag 0.5360 43 6 288 204
	plastic_bag 0.4381 87 0 200 74
=> ground truth of 201804023.jpg:
	tire 145 121 327 279
=> predict result of 201804023.jpg:
	tire 0.9984 137 125 321 274
	bowl 

	bowl 0.9165 148 131 174 152
	bucket 0.9946 184 124 293 287
=> ground truth of 2018081414.jpg:
	bucket 186 91 291 238
	bottle 107 20 162 87
	bowl 102 90 165 137
	bowl 174 126 199 178
=> predict result of 2018081414.jpg:
	bowl 0.8321 107 88 167 134
	bowl 0.2655 197 95 282 143
	bucket 0.9879 187 89 291 236
	bucket 0.9201 107 17 157 86
	plate 0.2973 176 178 292 241
=> ground truth of 2016050514.jpg:
	bucket 21 43 309 238
	box 6 3 144 65
=> predict result of 2016050514.jpg:
	bowl 0.7398 7 37 292 184
	box 0.4199 305 95 371 143
	box 0.3379 0 42 275 201
	bucket 0.9766 1 40 288 222
=> ground truth of 20130621.jpg:
	styrofoam 111 151 330 265
	bucket 132 128 271 186
	styrofoam 128 84 242 146
=> predict result of 20130621.jpg:
	box 0.4014 111 87 312 248
	water_tower 0.4173 128 103 277 221
	styrofoam 0.4550 147 107 237 225
	styrofoam 0.4198 97 121 348 266
=> ground truth of 201207103.jpg:
	bucket 65 113 141 255
	bucket 100 127 223 300
	bucket 197 152 374 300
=> predict result of 201207103.jpg:
	bo

	bowl 0.6032 36 4 274 226
	box 0.3525 40 0 241 285
	bucket 0.9661 54 4 264 268
	plate 0.3943 39 19 253 201
	tub 0.6159 44 33 237 299
	water_tower 0.6617 66 63 242 299
=> ground truth of 201403199.jpg:
	bowl 124 75 292 229
=> predict result of 201403199.jpg:
	bottle 0.2667 103 68 301 196
	bowl 0.9199 116 66 307 217
	tub 0.7093 102 71 263 234
	bucket 0.9549 355 95 398 167
	bucket 0.9269 118 90 296 244
=> ground truth of 201501141.jpg:
	bowl 83 145 149 177
	bowl 14 110 49 146
	box 210 90 239 124
	box 170 181 211 222
	box 208 181 252 223
	box 248 179 289 223
	box 234 119 264 144
	box 265 126 299 161
=> predict result of 201501141.jpg:
	tire 0.3432 84 152 144 179
	bowl 0.8625 86 147 147 177
	box 0.6129 212 137 245 157
	box 0.3045 235 144 262 159
	bucket 0.7649 12 107 48 138
=> ground truth of 2010081725.jpg:
	bucket 171 318 206 368
	bucket 205 331 253 372
=> predict result of 2010081725.jpg:
	bucket 0.9894 166 319 202 368
	bucket 0.9567 204 324 250 370
	styrofoam 0.4962 292 153 799 560
=> g

	bottle 0.3012 173 170 231 251
	styrofoam 0.3122 317 206 399 267
=> ground truth of 2015061820.jpg:
	bucket 141 80 288 295
=> predict result of 2015061820.jpg:
	bottle 0.8859 0 194 92 279
	tub 0.4786 119 97 323 299
	bucket 0.9967 140 93 284 299
=> ground truth of 2008112915.jpg:
	bucket 348 139 383 175
	bottle 415 129 472 155
=> predict result of 2008112915.jpg:
	bottle 0.3226 412 127 473 155
=> ground truth of 200811157.jpg:
	water_tower 51 45 148 198
=> predict result of 200811157.jpg:
	bowl 0.2841 55 68 148 179
	water_tower 0.9355 50 45 140 156
	bucket 0.8878 44 52 147 181
=> ground truth of 201404014.jpg:
	box 13 37 394 253
=> predict result of 201404014.jpg:
	box 0.9726 47 29 399 253
=> ground truth of 2013062013.jpg:
	bucket 103 41 301 225
	bucket 272 50 350 177
=> predict result of 2013062013.jpg:
	bowl 0.9526 93 19 301 211
	bucket 0.9715 94 52 310 224
	bucket 0.8213 283 62 350 165
	styrofoam 0.3665 112 19 320 211
	tire 0.4060 91 41 263 224
	tub 0.3969 115 55 317 204
=> ground t

	tire 0.6987 71 48 355 299
	bucket 0.9661 43 0 399 276
=> ground truth of 201403281.jpg:
	tire 193 186 320 237
	tire 65 113 173 205
	plastic_bag 110 64 283 177
	plastic_bag 122 166 400 241
	plastic_bag 21 2 203 132
=> predict result of 201403281.jpg:
	tire 0.6427 177 190 323 241
=> ground truth of 201606161.jpg:
	plate 78 2 331 285
	plate 63 220 259 300
=> predict result of 201606161.jpg:
	bowl 0.6021 72 222 248 293
	box 0.3103 38 25 356 228
	bucket 0.7141 72 224 227 299
	bucket 0.2721 43 0 382 236
	tire 0.4915 50 0 365 191
	tub 0.3315 75 224 245 299
=> ground truth of 201207274.jpg:
	box 182 162 322 254
=> predict result of 201207274.jpg:
	bucket 0.4959 278 170 318 241
=> ground truth of 201405012.jpg:
	styrofoam 184 126 302 215
	bucket 223 82 275 164
=> predict result of 201405012.jpg:
	bowl 0.7081 205 153 284 184
	bowl 0.6552 213 164 245 184
	box 0.2959 237 159 304 199
	bucket 0.9890 222 81 275 159
	bucket 0.5199 242 153 308 195
=> ground truth of 200904283.jpg:
	toilet 285 203 366 

	bucket 0.9707 154 188 188 242
	bucket 0.9545 130 176 169 234
	bucket 0.9423 186 208 214 242
	bucket 0.9317 211 203 234 239
	bucket 0.8440 231 207 248 231
	bucket 0.5901 114 169 152 230
	bucket 0.3779 266 141 349 238
=> ground truth of 200906151.jpg:
	bowl 9 106 43 131
=> predict result of 200906151.jpg:
=> ground truth of 201806285.jpg:
	tire 8 75 357 265
=> predict result of 201806285.jpg:
	aquarium 0.6630 78 76 336 270
	bowl 0.2621 0 38 383 267
	bucket 0.3415 55 37 338 278
	tire 0.9213 49 75 324 282
	water_tower 0.3590 57 112 315 275
=> ground truth of 200906251.jpg:
	tub 1 80 385 246
=> predict result of 200906251.jpg:
	box 0.5528 159 60 391 245
=> ground truth of 200907092.jpg:
	bucket 204 9 242 51
	box 86 70 122 92
	box 110 78 135 108
	plastic_bag 76 92 125 131
=> predict result of 200907092.jpg:
	bucket 0.2885 200 9 244 50
=> ground truth of 2010093013.jpg:
	bucket 77 200 107 262
=> predict result of 2010093013.jpg:
	bottle 0.2592 100 42 315 253
	plastic_bag 0.2590 60 48 355 267

	bottle 0.4461 127 128 141 163
	bucket 0.6222 157 156 197 201
	bucket 0.4257 56 8 150 74
=> ground truth of 201308071.jpg:
	box 1 68 315 299
=> predict result of 201308071.jpg:
	box 0.9211 43 57 320 299
	bucket 0.6493 0 58 371 256
	styrofoam 0.6887 25 94 317 299
	toilet 0.3353 30 0 260 213
	tub 0.8269 8 46 356 290
=> ground truth of 201602012.jpg:
	bucket 67 107 112 140
	tire 176 200 245 248
	tire 113 208 207 281
	tire 204 211 293 274
	tire 247 118 318 171
	tire 148 114 188 137
=> predict result of 201602012.jpg:
	tire 0.9392 147 112 195 139
	tire 0.7657 235 117 329 176
	plastic_bag 0.7005 26 155 92 222
	plastic_bag 0.5877 238 117 344 175
	plastic_bag 0.2858 84 194 244 299
=> ground truth of 2017062914.jpg:
	bucket 133 96 291 202
	styrofoam 244 33 400 150
=> predict result of 2017062914.jpg:
	bowl 0.8249 130 89 286 186
	box 0.4023 120 96 275 199
	bucket 0.8452 128 101 287 202
	bucket 0.6588 221 3 258 70
	styrofoam 0.7444 233 23 399 154
=> ground truth of 201802265.jpg:
	bucket 49 207 1

	bowl 0.3864 67 53 295 187
	bucket 0.9426 58 51 314 200
	plate 0.7036 73 56 320 164
=> ground truth of 2009041511.jpg:
	water_tower 60 1 248 150
=> predict result of 2009041511.jpg:
	styrofoam 0.2923 303 248 362 299
=> ground truth of 2013072910.jpg:
	bucket 164 59 229 142
	box 156 92 256 226
=> predict result of 2013072910.jpg:
	bucket 0.9105 167 48 224 141
	bucket 0.2843 149 140 255 228
=> ground truth of 201706221.jpg:
	bucket 61 135 275 225
	bucket 206 181 375 277
	bowl 1 188 204 300
	plastic_bag 130 30 246 97
=> predict result of 201706221.jpg:
	bowl 0.9745 210 179 363 272
	bowl 0.7651 0 182 197 299
	bowl 0.5165 31 128 299 220
	box 0.5121 46 133 242 214
	bucket 0.8853 206 189 366 291
	bucket 0.5602 46 129 294 227
	plastic_bag 0.7321 130 31 252 104
	plastic_bag 0.6333 17 188 228 286
	plate 0.8234 219 177 371 256
	plate 0.7995 0 189 223 299
	plate 0.3531 31 128 289 208
	tire 0.7794 0 100 140 140
=> ground truth of 201405149.jpg:
	bowl 71 3 384 284
	bowl 11 214 89 295
=> predict resu

	bowl 0.4881 99 92 196 172
	bucket 0.9936 89 184 159 278
	bucket 0.9757 221 66 290 124
	bucket 0.9741 99 93 190 201
=> ground truth of 201011115.jpg:
	box 251 129 388 218
	box 204 130 266 205
	bucket 249 327 298 387
	plastic_bag 133 192 463 324
=> predict result of 201011115.jpg:
	box 0.3435 434 251 547 358
	bucket 0.7785 251 331 296 386
	bucket 0.4235 203 135 272 200
	bucket 0.3258 448 273 521 361
	plastic_bag 0.3601 153 152 493 434
=> ground truth of 2010112614.jpg:
	box 293 157 367 208
	box 284 132 346 174
	box 281 111 336 143
	box 280 95 328 119
	box 279 80 327 104
	box 162 66 208 87
	box 150 80 202 100
	box 131 113 188 142
=> predict result of 2010112614.jpg:
	bucket 0.9347 145 62 208 126
=> ground truth of 201209103.jpg:
	bucket 126 113 309 299
	bucket 127 18 185 89
	bucket 88 79 162 143
	bucket 61 129 150 207
	bucket 180 44 235 71
	bucket 75 202 146 278
=> predict result of 201209103.jpg:
	bowl 0.9495 127 105 308 264
	bowl 0.9407 67 121 148 195
	bowl 0.3467 149 59 258 123
	tub 0